In [1]:
import pandas as pd
import math
import logging
from sklearn.model_selection import TimeSeriesSplit
from copy import deepcopy
from prophet import Prophet
from prophet.serialize import model_to_json


from eval import my_grid_search_cv

logging.getLogger("prophet").setLevel(logging.CRITICAL)
logging.getLogger("cmdstanpy").setLevel(logging.CRITICAL)

logging.getLogger("fbprophet.forecaster").propagate = False

# from tqdm import tqdm
# tqdm.pandas()

In [2]:
import os

if os.getcwd() != "/root/restaurants":
    os.chdir("..")
os.getcwd()

'/root/restaurants'

In [3]:
data = pd.read_csv("data/processed/data.csv")
data.head()

,store_id,date,visitors,day_of_week,holiday_flg,genre_name,area_name,latitude,longitude,year,...,lag_day_of_week_140,lag_holiday_flg_140,lag_area_genre_day_of_week_280,lag_area_genre_holiday_flg_280,lag_day_of_week_280,lag_holiday_flg_280,lag_area_genre_day_of_week_350,lag_area_genre_holiday_flg_350,lag_day_of_week_350,lag_holiday_flg_350
0,air_00a91d42b08b08d9,2016-07-01,35,Friday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,2016,...,35.0,35.000000,25.000000,25.000000,35.0,35.000000,25.000000,25.000000,35.0,35.000000
1,air_00a91d42b08b08d9,2016-07-02,9,Saturday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,2016,...,9.0,22.000000,16.500000,20.750000,9.0,22.000000,16.500000,20.750000,9.0,22.000000
2,air_00a91d42b08b08d9,2016-07-04,20,Monday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,2016,...,20.0,21.333333,14.000000,17.875000,20.0,21.333333,14.000000,17.875000,20.0,21.333333
3,air_00a91d42b08b08d9,2016-07-05,25,Tuesday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,2016,...,25.0,22.250000,20.333333,18.366667,25.0,22.250000,20.333333,18.366667,25.0,22.250000
4,air_00a91d42b08b08d9,2016-07-06,29,Wednesday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,2016,...,29.0,23.600000,14.750000,17.763889,29.0,23.600000,14.750000,17.763889,29.0,23.600000


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252108 entries, 0 to 252107
Data columns (total 36 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   store_id                        252108 non-null  object 
 1   date                            252108 non-null  object 
 2   visitors                        252108 non-null  int64  
 3   day_of_week                     252108 non-null  object 
 4   holiday_flg                     252108 non-null  int64  
 5   genre_name                      252108 non-null  object 
 6   area_name                       252108 non-null  object 
 7   latitude                        252108 non-null  float64
 8   longitude                       252108 non-null  float64
 9   year                            252108 non-null  int64  
 10  month                           252108 non-null  int64  
 11  day                             252108 non-null  int64  
 12  lag_area_genre_d

In [ ]:
cat_cols = ["store_id"]

In [ ]:
# from tqdm.auto import tqdm

# tqdm.pandas()

# small = []

# directory = "models/catboost_for_each_store"
# os.makedirs(directory, exist_ok=True)

# models = dict()

# param_grid = {
#     "changepoint_prior_scale": [0.001, 0.01, 0.1, 0.5],
#     "seasonality_prior_scale": [0.01, 0.1, 1, 10],
#     "seasonality_mode": ["additive", "multiplicative"],
#     "holidays": [holidays],
# }

# for store_id, store_data in tqdm(data.groupby("air_store_id")):
    
#     # print(store_id, store_data.shape[0])

#     file_path = os.path.join(directory, f"model_{store_id}.json")

#     if os.path.exists(file_path):
#         continue

#     X = store_data[["visit_date"]].reset_index(drop=True)
#     y = store_data[["visitors"]].reset_index(drop=True)

#     n_splits = max(min(math.trunc((store_data.shape[0] - 5) / 30), 5), 2)

#     cv = TimeSeriesSplit(n_splits=n_splits, gap=3, test_size=30)

#     if store_data.shape[0] <= 65:
#         small.append(store_id)
#         cv = TimeSeriesSplit(
#             n_splits=2, gap=1, test_size=int(store_data.shape[0] * 0.2)
#         )

#     params = my_grid_search_cv(Prophet, X, y, param_grid, cv=cv, n_jobs=5)

#     models[store_id] = Prophet(**params)

#     df = pd.concat([X, y], axis=1)
#     df = df.rename(columns={"visit_date": "ds", "visitors": "y"})
#     models[store_id].fit(df)

#     with open(file_path, "w") as fout:
#         fout.write(model_to_json(models[store_id]))